In [1]:

%load_ext autoreload
%autoreload 2

In [45]:
data = QM9(r"../data/qm9/")

In [53]:
from rdkit import Chem

import pandas as pd
import numpy as np

path = "qm9_smiles.csv"

suppl = Chem.SmilesMolSupplier(path)

df = pd.read_csv(path)

df["target1"] = np.random.randn((len(df)))
df["target2"] = np.random.randn((len(df)))

df.set_index("smiles",inplace=True)

df.to_csv("qm9_target.csv")

## Testing SmilesDataset

In [7]:
from smiles_dataset import SmilesDataset
from smiles_lightning_data_module import SmilesDataModule
from lightning_model import LightningClassicGNN
import pytorch_lightning as pl
from torch_geometric.transforms import distance
from torch_geometric.loader import DataLoader
import os
import torch
# making sure we are as determinstic as possibe
torch.use_deterministic_algorithms(True)
import numpy
import random

! rm -rf ../data/test/processed
! rm -rf ../data/test_hydrogen/processed


seed=42
## pytorch lighting takes of seeding everything
pl.seed_everything(seed=seed, workers=True)
# create from csv a pytorch dataset
dataset = SmilesDataset(root=r"../data/test", filename="qm9_target.csv", add_hydrogen=True, transform=distance.Distance())
# from torch dataset, create lightning data module to make sure training splits are always done the same ways
data_module = SmilesDataModule(dataset=dataset, seed=seed)


num_node_features = data_module.num_node_features
num_edge_features= data_module.num_edge_features

gnn_model = LightningClassicGNN(classification=False, output_dim=2, num_node_features=num_node_features, num_edge_features=num_edge_features)

num_epochs=1


# from pytorch_ligthing import loggers
# logger = loggers.WandbLogger()

# default root dir is where the logs and weights are logged
# useful when debugging is limit_train_batches
# by default uses TensorBoardLogger, can be configured 
# Plugins allows us to connect to arbitrary cluster
# can set max_epochs
# can use precision to specify number of bit floating points to reduce memory footprint ()
# can use accumulate_grad_batches to speed-up training too
trainer = pl.Trainer(deterministic=True, auto_lr_find=True, default_root_dir=os.getcwd(), precision="bf16", max_epochs=num_epochs)



Global seed set to 42
Processing...
100%|██████████| 100/100 [00:00<00:00, 105.91it/s]
Done!
Global seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [8]:
# tune to find the learning rate
trainer.tune(gnn_model,datamodule=data_module)

/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LR finder stopped early after 2 steps due to diverging loss.
Failed to compute suggestion for learning rate because there are not enough points. Increase the loop iteration limits or the size of your dataset/dataloader.
Restoring states from the checkpoint path at /home/harold/epfl/chem-ml-repr/code/.lr_find_ef32d5bb-2f32-433c-adc9-58cebf7beeee.ckpt


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7fcce733e460>}

In [5]:

# we can resume from a checkpoint using trainer.fit(ckpth_path="some/path/to/my_checkpoint.ckpt")
trainer.fit(gnn_model, datamodule=data_module)

(5, 5)

In [2]:
from datasets import QM9Dataset
import os

root= "../data/qm9"

data = QM9Dataset(root=root)


Processing...
  0%|          | 49/133885 [00:00<16:40, 133.77it/s]


RuntimeError: Class values must be smaller than num_classes.